In [ ]:
#INPUT: assignments.csv, [optional] area.csv
#OUTPUT: counts.h5ad


#Read assignments

#Generate blank, labelled count matrix

#Populate matrix using assignments

#Calculate area via alpha shapes if none provided

#Normalize by area or by total counts